# Imports and necessary assignments

In [4]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import requests, json, sqlite3, time
#from msedge.selenium_tools import EdgeOptions

In [5]:
headers = {"accept-encoding": "gzip, deflate, br",
           "accept-language": "en-US,en;q=0.9",
           "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36 Edg/90.0.818.56"}

# Find product links and save links to file

In [7]:
men_links = []
all_men_items = 799
for page in range(1, all_men_items//18+2):
    url = f"https://www.metroshoes.net/mana_layerednavigationajax/products/?_id=3&_route=catalog_category_view&p={page}"
    page_html = bs(requests.get(url, headers=headers).text, 'html.parser')
    for a_tag in page_html.findAll('a', {'class': 'product-item-link'}):
        men_links.append(a_tag['href'])
    #print(page, len(men_links))
    
men_links = list(set(men_links))
print(len(men_links))

1661


In [8]:
with open('metroshoes_prod_links.csv', 'w') as f:
    for link in men_links:
            f.write(link+"\n")

In [9]:
women_links = []
all_women_items = 1223
for page in range(1, all_women_items//18+2):
    url = f"https://www.metroshoes.net/mana_layerednavigationajax/products/?_id=4&_route=catalog_category_view&p={page}"
    page_html = bs(requests.get(url, headers=headers).text, 'html.parser')
    for a_tag in page_html.findAll('a', {'class': 'product-item-link'}):
        women_links.append(a_tag['href'])
    print(page, len(women_links))

women_links = list(set(women_links))
print(len(women_links))

1 108
2 216
3 324
4 432
5 540
6 648
7 756
8 864
9 972
10 1080
11 1188
12 1296
13 1404
14 1512
15 1620
16 1728
17 1836
18 1944
19 2052
20 2160
21 2268
22 2376
23 2484
24 2592
25 2700
26 2808
27 2916
28 3024
29 3132
30 3240
31 3348
32 3456
33 3564
34 3672
35 3780
36 3888
37 3996
38 4104
39 4212
40 4244
41 4276
42 4308
43 4340
44 4372
45 4404
46 4436
47 4468
48 4500
49 4532
50 4564
51 4596
52 4628
53 4660
54 4692
55 4724
56 4756
57 4788
58 4820
59 4852
60 4884
61 4916
62 4948
63 4980
64 5012
65 5044
66 5076
67 5108
68 5140
2122


In [10]:
with open('metroshoes_prod_links.csv', 'a') as f:
    for link in women_links:
            f.write(link+"\n")

In [11]:
accessories_links = []
all_accessories_items = 319
for page in range(1, all_accessories_items//18+2):
    url = f"https://www.metroshoes.net/mana_layerednavigationajax/products/?_id=7&_route=catalog_category_view&p={page}"
    page_html = bs(requests.get(url, headers=headers).text, 'html.parser')
    for a_tag in page_html.findAll('a', {'class': 'product-item-link'}):
        accessories_links.append(a_tag['href'])
    print(page, len(accessories_links))

accessories_links = list(set(accessories_links))
print(len(accessories_links))

1 108
2 216
3 324
4 432
5 540
6 648
7 756
8 864
9 972
10 1080
11 1188
12 1296
13 1404
14 1502
15 1600
16 1698
17 1796
18 1894
751


In [ ]:
with open('metroshoes_prod_links.csv', 'a') as f:
    for link in accessories_links:
            f.write(link+"\n")

In [12]:
bags_links = []
all_bags_items = 482
for page in range(1, all_bags_items//18+2):
    url = f"https://www.metroshoes.net/mana_layerednavigationajax/products/?_id=212&_route=catalog_category_view&p={page}"
    page_html = bs(requests.get(url, headers=headers).text, 'html.parser')
    for a_tag in page_html.findAll('a', {'class': 'product-item-link'}):
        bags_links.append(a_tag['href'])
    print(page, len(bags_links))

bags_links = list(set(bags_links))
print(len(bags_links))

1 108
2 216
3 324
4 432
5 540
6 648
7 756
8 864
9 972
10 1080
11 1188
12 1296
13 1404
14 1512
15 1620
16 1658
17 1696
18 1734
19 1772
20 1810
21 1848
22 1886
23 1924
24 1962
25 2000
26 2038
27 2076
829


In [13]:
with open('metroshoes_prod_links.csv', 'a') as f:
    for link in bags_links:
            f.write(link+"\n")

### Extract saved links from file

In [14]:
all_links = []

with open("metroshoes_prod_links.csv","r") as f:
    for line in f:
        all_links.append(line.strip())

all_links = list(set(all_links))
len(all_links)

4612

In [ ]:
main_link

# Functions
## Function to get each product details

In [16]:
def get_details(main_link):
    try:
        site_content = requests.get(main_link, headers=headers).text
        site_html    = bs(site_content, 'html.parser')
        site_json = json.loads(site_html.findAll('script')[46].contents[0])
        site_json2 = json.loads(site_html.findAll('script')[40].contents[0][97:-95])['jsonConfig']
        
        #breadcrumb = site_html.find('div', {'class' :'breadcrumbs'}).text.lower()
        sizes = [(i['label']) for i in site_json2['attributes']['144']['options']]
        imgs = [img['href'] for img in site_html.findAll('div', {'class': 'MagicToolboxContainer'})[0].findAll('a')]
        
        if "women" in main_link: gender = "Women"
        elif "men" in main_link: gender = "Men"
        else: gender = "Unisex"
        if "accessories" in main_link: category = "Accessories"
        elif "bags" in main_link: category = "Bags"
        else: category = "Shoes"

        product_details = {}
        product_details['link']          = main_link
        product_details['name']          = site_json['name']
        product_details['brand']         = site_json['brand']
        product_details['category']      = category
        product_details['avail_size']    =  ", ".join(sizes)
        product_details['price']         = site_json2['prices']['finalPrice']['amount']
        product_details['mrp']           = site_json2['prices']['oldPrice']['amount']
        product_details['gender']        = gender
        product_details['desc']          = site_json['description']
        product_details['primary_img']   = imgs[0]
        product_details['secondary_img'] = ", ".join(imgs[1:])
        
        return product_details
    except Exception as e:
        print(e, end="---")
        return None

## Function to add each product details to db

In [17]:
def add_to_db(product_details):
    try:
        cursor.execute("INSERT INTO product_details(Website, Product_Link, Product_Name, Product_Brand, Product_Category, Size_Avail, Price, MRP, Gender, Description, Primary_Image_Links, Secondary_Image_Links) VALUES(?,?,?,?, ?,?,?,?, ?,?,?,?)",
                       ("https://www.metroshoes.net/",
                        product_details['link'],
                        product_details['name'],
                        product_details['brand'],
                        product_details['category'],
                        product_details['avail_size'],
                        product_details['price'],
                        product_details['mrp'],
                        product_details['gender'],
                        product_details['desc'],
                        product_details['primary_img'],
                        product_details['secondary_img'])
                      )
        conn.commit()
        print("Added to db")
    except Exception as e:
        print(e)
        pass

# Create db

In [18]:
conn = sqlite3.connect("metroshoes.db")
cursor = conn.cursor()
try:
    cursor.execute("""CREATE TABLE product_details(
                    Website varchar(40) not null,
                    Product_Link text PRIMARY KEY,
                    Product_Name varchar(50) not null,
                    Product_Brand varchar(50) not null,
                    Product_Category varchar(50),
                    Size_Avail varchar(20) not null,
                    Price int not null,
                    MRP int not null,
                    Gender varchar(15) not null,
                    Description text not null,
                    Primary_Image_Links text not null,
                    Secondary_Image_Links text not null,
                    Affiliate_Link text
                )""")
    conn.commit()
except Exception as e:
    print(e)
    pass

# Get product details and save to db directly

### Save links to file / extract links from file (to avoid collecting links again in case of notebook crash)

In [ ]:
#all_details=[]
for i, link in enumerate(all_links[2584:]):
    print(i+2584, end=": ")
    data=get_details(link)
    add_to_db(data)
    #all_details.append(data)
    print(link, end="\n\n") 

2584: Expecting value: line 2 column 1 (char 1)---'NoneType' object is not subscriptable
https://www.metroshoes.net/men-shoes/metro-19-5172-black-formal-moccasin.html

2585: Expecting value: line 2 column 1 (char 1)---'NoneType' object is not subscriptable
https://www.metroshoes.net/women-shoes/metro-32-842-chikoo-casual-slip-ons.html

2586: Expecting value: line 2 column 1 (char 1)---'NoneType' object is not subscriptable
https://www.metroshoes.net/women-shoes/mochi-35-4325-chikoo-party-sandals.html

2587: Expecting value: line 2 column 1 (char 1)---'NoneType' object is not subscriptable
https://www.metroshoes.net/women-shoes/walkway-32-1097-maroon-casual-slip-ons.html

2588: Expecting value: line 2 column 1 (char 1)---'NoneType' object is not subscriptable
https://www.metroshoes.net/women-shoes/mochi-32-9184-white-casual-slip-ons.html

2589: Expecting value: line 2 column 1 (char 1)---'NoneType' object is not subscriptable
https://www.metroshoes.net/women-shoes/metro-35-3665-antic-go

#  --OR-- get all product details first and then add all to db 

In [ ]:
all_details=[]
for i, link in enumerate(all_links):
    print(i, end=": ")
    data=get_details(link)
    #add_to_db(data)
    all_details.append(data)
    print(link, end="\n\n") 

In [ ]:
for i, product_detail in enumerate(all_details):
    print(i, end=": ")
    #data=get_details(link)
    add_to_db(data)
    #all_details.append(data)
    print(link, end="\n\n") 

# Check if data added in db and close connection

In [ ]:
cursor.execute("select Product_Link from product_details")
rows = cursor.fetchall()
len(rows)

In [ ]:
conn.close()